# ADES

This notebook demostrates usage of the EODH ADES API using `pyeodh`.

EODH provides Application Deployment & Execution Service - ADES, to which you can deploy workflows and execute parametrised processing jobs. Pyeodh provides an interface to simplify interaction with ADES from python scripts.

First we need to instantiate pyeodh client and get the ADES entrypoint.

Note: This API requires authentication credentials to be provided by the user (in this case read from environment variables). This is a subject to change as the hub is implementing proper IAM solution.

In [1]:
from requests import HTTPError
import os
from pprint import pp

import pyeodh

username = os.getenv("ADES_USER")
token = os.getenv("ADES_TOKEN")
s3_token = os.getenv("ADES_S3_TOKEN")

client = pyeodh.Client(username=username, token=token, s3_token=s3_token)
ades = client.get_ades()

Processes (or workflows) are predefined applications which can be parametrised and executed by users. To get a list of currently available processes in our user workspace call `Ades.get_processes()` method:

In [2]:
for p in ades.get_processes():
    print(p.id)

display
echo
convert-stac
water-bodies
convert-url


You can fetch a specific workflow if you know it's ID using `Ades.get_process()` method. The `Process` object also contains metadata giving us more information about the process and how to execute it, for example the schema of inputs we can use to parametrise the process or output schema.

In [3]:
convert_url_proc = ades.get_process("convert-url")

pp(convert_url_proc.inputs_schema)
pp(convert_url_proc.outputs_schema)


{'fn': {'title': 'the operation to perform',
        'description': 'the operation to perform',
        'schema': {'type': 'string', 'default': 'resize', 'nullable': True}},
 'size': {'title': 'the percentage for a resize operation',
          'description': 'the percentage for a resize operation',
          'schema': {'type': 'string', 'default': '50%', 'nullable': True}},
 'url': {'title': 'the image to convert',
         'description': 'the image to convert',
         'schema': {'type': 'string',
                    'default': 'https://eoepca.org/media_portal/images/logo6_med.original.png',
                    'nullable': True}}}
{'converted_image': {'title': 'converted_image',
                     'description': 'None',
                     'extended-schema': {'oneOf': [{'allOf': [{'$ref': 'http://zoo-project.org/dl/link.json'},
                                                              {'type': 'object',
                                                               'properties

Get list of previously executed jobs

In [4]:
for j in ades.get_jobs():
    print(j.id, j.process_id, j.status)

45cfba32-4ad1-11ef-a8c6-66f4c7b2b89b convert-url failed
9dcfec66-4ad1-11ef-ab61-66f4c7b2b89b convert-url failed
d10202bc-53ec-11ef-8d4c-06180dac6ed1 water-bodies successful


Only one process with the same ID can exist. To demonstrate deploying a process further down in this notebook, we first need to undeploy `convert-url`. Note that attempting to delete a non-existent process will result in 4xx http status code.

In [5]:
try:
    ades.get_process("convert-url").delete()
except HTTPError:
    print("Process not found, no need to undeploy.")

Let's deploy the `convert-url` process again. There are 2 ways we can provide the CWL file - either referencing the file by URL or by passing the CWL file content directly. Note that `Ades.deploy_process()` will fail if we try to create a process with ID that already exists. If we want to update an existing process, we should use `Process.update()` method instead. Both methods can handle URL or CWL YAML. In this example we deploy a process referencing by URL and then update it by passing the new CWL YAML content directly. Also note that when updating a worklow you need to provide the entire workflow, the API does not support partial updates (e.g. to change the description we need to provide the entire workflow again).

In [6]:
convert_url_proc = ades.deploy_process(
    cwl_url="https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/deploy/samples/requests/processing/convert-url-app.cwl"
)
print(convert_url_proc.id, convert_url_proc.description)

convert-url Convert URL


In [7]:
cwl_yaml = """cwlVersion: v1.0
$namespaces:
  s: https://schema.org/
s:softwareVersion: 0.1.2
schemas:
  - http://schema.org/version/9.0/schemaorg-current-http.rdf
$graph:
  # Workflow entrypoint
  - class: Workflow
    id: convert-url
    label: convert url app
    doc: Convert URL YAML
    requirements:
      ResourceRequirement:
        coresMax: 1
        ramMax: 1024
    inputs:
      fn:
        label: the operation to perform
        doc: the operation to perform
        type: string
        default: "resize"
      url:
        label: the image to convert
        doc: the image to convert
        type: string
        default: "https://eoepca.org/media_portal/images/logo6_med.original.png"
      size:
        label: the percentage for a resize operation
        doc: the percentage for a resize operation
        type: string
        default: "50%"
    outputs:
      - id: converted_image
        type: Directory
        outputSource:
          - convert/results
    steps:
      convert:
        run: "#convert"
        in:
          fn: fn
          url: url
          size: size
        out:
          - results
  # convert.sh - takes input args `--url`
  - class: CommandLineTool
    id: convert
    requirements:
      ResourceRequirement:
        coresMax: 1
        ramMax: 512
    hints:
      DockerRequirement:
        dockerPull: eoepca/convert:latest
    baseCommand: convert.sh
    inputs:
      fn:
        type: string
        inputBinding:
          position: 1
      url:
        type: string
        inputBinding:
          position: 2
          prefix: --url
      size:
        type: string
        inputBinding:
          position: 3
    outputs:
      results:
        type: Directory
        outputBinding:
          glob: .
"""

convert_url_proc.update(cwl_yaml=cwl_yaml)
print(convert_url_proc.id, convert_url_proc.description)

convert-url Convert URL YAML


Let's execute our deployed process. We need to provide inputs as a dictionary in this format, see `Process.inputs_schema` property for inputs this particular workflow is expecting.

In [4]:
convert_url_job = convert_url_proc.execute(
    {
        "fn": "resize",
        "url": "https://eoepca.org/media_portal/images/logo6_med.original.png",
        "size": "50%",
    }
)

print(convert_url_job.id, convert_url_job.status, convert_url_job.message)

f280956e-58ad-11ef-8ed7-8635b5d444bf running ZOO-Kernel accepted to run your service!


The job should now be running. Call `Job.refresh()` method to get the most up-to-date status and interrogate `Job.status` and `Job.message` properties. Note that these properties only hold the latest response from the API, and don't keep any historical records.

In [7]:
convert_url_job.refresh()
print(convert_url_job.id, convert_url_job.status, convert_url_job.message)

f280956e-58ad-11ef-8ed7-8635b5d444bf running processing environment created, preparing execution


We can continually poll the job using a simple loop and print status and message udpates like so:

In [8]:
from pyeodh.ades import AdesJobStatus
import time


old_status = ""
old_message = ""
while convert_url_job.status == AdesJobStatus.RUNNING.value:
    time.sleep(2)
    convert_url_job.refresh()
    if convert_url_job.status != old_status:
        print("\n")
        print(f"Status: {convert_url_job.status}")
    if convert_url_job.message != old_message:
        print(f"Message: {convert_url_job.message}")

    old_status = convert_url_job.status
    old_message = convert_url_job.message



Status: running
Message: processing environment created, preparing execution
Message: execution submitted
Message: delivering outputs, logs and usage report
Message: Post-execution hook


Status: successful
Message: ZOO-Kernel successfully run your service!


After the job has finished successfully, we can view the results, where the data files are referenced by assets

In [2]:
results = convert_url_job.get_result_items()
for res in results:
    print(res.id, res.assets)

logo6_med.original-resize-1723653500.022521511 {'logo6_med.original-resize': <Asset href=https://figi44.workspaces.test.eodhp.eco-ke-staging.com/files/eodhp-test-workspaces1/processing-results/cat_8e9c782e-5a5b-11ef-ad70-8635b5d444bf/col_8e9c782e-5a5b-11ef-ad70-8635b5d444bf/logo6_med.original-resize.png>}
